In [1]:
import time
import pandas as pd

from src.domain.Collection import LiveJobCollection
from decimal import Decimal
from src.domain.Query import JobQuery

from src.DataFrameAnalyses import DataFrameChecker
from src.solvers.heuristics.giffler_thompson import solve as solve_gt

In [2]:
jobs = JobQuery.get_by_source_name_max_util_and_lt_arrival(
    source_name="Fisher and Thompson 10x10",
    max_bottleneck_utilization = Decimal("0.90"),
    arrival_limit= 60*24*2 # 2 days
)
#jobs = [job for job in all_jobs if job.earliest_start <=timespan]

jobs_collection = LiveJobCollection(jobs)

In [3]:
for job in list(jobs_collection.values())[:2]:
    print(f"{job.id = } {job.earliest_start = } {job.due_date = }")
    for operation in job.operations:
        print(f" {operation.position_number = } {operation.machine_name = } {operation.duration = }")

job.id = '01-09500-0000' job.earliest_start = 1440 job.due_date = 3326
 operation.position_number = 0 operation.machine_name = 'M02' operation.duration = 31
 operation.position_number = 1 operation.machine_name = 'M00' operation.duration = 86
 operation.position_number = 2 operation.machine_name = 'M01' operation.duration = 46
 operation.position_number = 3 operation.machine_name = 'M05' operation.duration = 74
 operation.position_number = 4 operation.machine_name = 'M04' operation.duration = 32
 operation.position_number = 5 operation.machine_name = 'M06' operation.duration = 88
 operation.position_number = 6 operation.machine_name = 'M08' operation.duration = 19
 operation.position_number = 7 operation.machine_name = 'M09' operation.duration = 48
 operation.position_number = 8 operation.machine_name = 'M07' operation.duration = 36
 operation.position_number = 9 operation.machine_name = 'M03' operation.duration = 79
job.id = '01-09500-0001' job.earliest_start = 1440 job.due_date = 364

In [10]:
from src.DataFrameAnalyses import DataFrameChecker
from src.solvers.heuristics.giffler_thompson import solve as solve_gt

+ No machine conflicts found.
+ All job operations are scheduled in non-overlapping, correct sequence.
+ All operations start at or after the earliest allowed time.
+ All durations match the difference between start and end.


True

In [11]:
schedule_collection = solve_gt(jobs_collection, priority_rule="EDD")
df_schedule = schedule_collection.to_operations_dataframe()
df_schedule

Giffler-Thompson-Algorithmus mit Prioritätsregel "EDD" gestartet ...


Plan-Informationen:
  Anzahl Jobs        : 74
  Anzahl Maschinen   : 10
  Makespan           : 7471
  Laufzeit           : ~0.01 Sekunden


,Job,Routing_ID,Operation,Machine,Start,Processing Time,End,Arrival,Ready Time,Due Date
0,01-09500-0005,01-02,0,M01,1440,91,1531,396,1440,2639
1,01-09500-0005,01-02,1,M00,1565,85,1650,396,1440,2639
2,01-09500-0005,01-02,2,M03,1650,39,1689,396,1440,2639
3,01-09500-0005,01-02,3,M02,1704,74,1778,396,1440,2639
4,01-09500-0005,01-02,4,M08,1778,90,1868,396,1440,2639
...,...,...,...,...,...,...,...,...,...,...
735,01-09500-0066,01-03,5,M08,7112,52,7164,4088,4320,8434
736,01-09500-0066,01-03,6,M07,7223,85,7308,4088,4320,8434
737,01-09500-0066,01-03,7,M03,7308,98,7406,4088,4320,8434
738,01-09500-0066,01-03,8,M09,7406,22,7428,4088,4320,8434


In [13]:
DataFrameChecker.check_core_schedule_constraints(df_schedule)
DataFrameChecker.is_duration_correct(df_schedule)

+ No machine conflicts found.
+ All job operations are scheduled in non-overlapping, correct sequence.
+ All operations start at or after the earliest allowed time.
+ All durations match the difference between start and end.


True

In [14]:
df_jobs_metrics = schedule_collection.to_jobs_metrics_dataframe()
df_jobs_metrics

,Job,Routing_ID,Last Operation,End,Arrival,Ready Time,Due Date,Total Processing Time,Flowtime,Lateness,Tardiness,Earliness
0,01-09500-0005,01-02,9,2086,396,1440,2639,568,646,-553,0,553
1,01-09500-0004,01-04,9,1919,196,1440,3015,393,479,-1096,0,1096
2,01-09500-0002,01-01,9,2043,112,1440,3135,510,603,-1092,0,1092
3,01-09500-0000,01-07,9,2381,0,1440,3326,539,941,-945,0,945
4,01-09500-0012,01-08,9,2300,970,1440,3294,597,860,-994,0,994
...,...,...,...,...,...,...,...,...,...,...,...,...
69,01-09500-0059,01-04,9,7282,3786,4320,7553,393,2962,-271,0,271
70,01-09500-0061,01-02,9,6897,3935,4320,6785,568,2577,112,112,0
71,01-09500-0063,01-06,9,7001,3969,4320,6857,416,2681,144,144,0
72,01-09500-0064,01-00,9,7252,4031,4320,7689,395,2932,-437,0,437


In [15]:
from src.DataFrameAnalyses import DataFrameMetricsAnalyser

DataFrameMetricsAnalyser.get_jobs_metrics_aggregated(df_jobs_metrics, column="Tardiness")

Tardiness_Interval,0,0 - 60,60 - 120,>120
count,69,2,2,1


In [ ]:
from matplotlib import pyplot as plt
from src.DataFrameAnalyses import DataFramePlotGenerator

DataFramePlotGenerator.get_gantt_chart_figure(df_schedule, perspective="Machine")
plt.show()